This is the notebook that was used in the first class of data science.
We first start by loading the packages we are going to use for the session

In [3]:
import pandas as pd #dealing with dataframes
import numpy as np #dealing with datastructures
import datetime as dt #d4ealing with date times

This data is available on the github page and was shared to us by EVA.
In order to read the data into python we use pandas read csv method as shown below

In [5]:
accidents = pd.read_csv("data/2011_Traffic_Incidences_From_Desinventar.csv")

We call on the head function to be able to see the top 5 records that gives us an impression of what the data looks like

In [7]:
accidents.head(5)

,Serial,Event,Code_County,County,Code_District,District,Code_Division,Division,Date_YMD,Comments,...,Affected,Relocated,Evacuated,Losses_$USD,Losses_$Local,Education_centers,Hospitals,Damages_in_crops_Ha,Lost_Cattle,OBJECTID
0,Serial,Event,Code County,County,Code District,District,Code Division,Division,Date (YMD),Comments,...,Affected,Relocated,Evacuated,Losses $USD,Losses $Local,Education centers,Hospitals,Damages in crops Ha.,Lost Cattle,0
1,11,TRAFFIC ACCIDENT,13,KIAMBU,1301,KIAMBU,130104,LARI,11/09/2011 08:00:00 AM +0000,NaN,...,0,0,0,0,0,0,0,0,0,1
2,29,TRAFFIC ACCIDENT,30,NAIROBI,3001,NAIROBI,300106,MAKADARA,01/01/2011 08:00:00 AM +0000,NaN,...,0,0,0,0,0,0,0,0,0,2
3,35,TRAFFIC ACCIDENT,12,KERICHO,1203,BURET,120303,KONOIN,01/01/2011 08:00:00 AM +0000,NaN,...,0,0,0,0,0,0,0,0,0,3
4,20,TRAFFIC ACCIDENT,23,MAKUENI,2305,MAKUENI,230504,KASIKEU,01/30/2011 08:00:00 AM +0000,NaN,...,0,0,0,0,0,0,0,0,0,4


Shape allows us to tell how many rows and columns the data has

In [9]:
accidents.shape

(1355, 33)

Columns lists all the columns/variables the data has

In [12]:
accidents.columns

Index(['Serial', 'Event', 'Code_County', 'County', 'Code_District', 'District',
       'Code_Division', 'Division', 'Date_YMD', 'Comments', 'Cause',
       'Description_of_Cause', 'Source', 'Magnitude', 'GLIDEnumber',
       'Other_sectors', 'DataCards', 'Deaths', 'Injured', 'Missing',
       'Houses_Destroyed', 'Houses_Damaged', 'Victims', 'Affected',
       'Relocated', 'Evacuated', 'Losses_$USD', 'Losses_$Local',
       'Education_centers', 'Hospitals', 'Damages_in_crops_Ha', 'Lost_Cattle',
       'OBJECTID'],
      dtype='object')

Without a data dictionary as was the case with this data we have to look through columns one at a time to make inferences what they are about.

In [41]:
accidents.Date_YMD.head(10)

0                      Date (YMD)
1    11/09/2011 08:00:00 AM +0000
2    01/01/2011 08:00:00 AM +0000
3    01/01/2011 08:00:00 AM +0000
4    01/30/2011 08:00:00 AM +0000
5    01/01/2011 08:00:00 AM +0000
6    01/30/2011 08:00:00 AM +0000
7    01/01/2011 08:00:00 AM +0000
8    01/30/2011 08:00:00 AM +0000
9    01/01/2011 08:00:00 AM +0000
Name: Date_YMD, dtype: object

From the head fuction we can see that the first row contains column names. We remove this as below:
Notice that we have assigned the new data to a new dataframe so that we can be able to go back whenever we realise me made a mistake

In [48]:
accidents2 = accidents[1:]
accidents2.shape

(1354, 33)

Next we separate the data parameters using regex so that we have dates only which is important in having a better understanding of the times at which the data was collected.

In [51]:
accidents2[["dateonly","deleteall"]]=accidents2.Date_YMD.str.split(" ", 1, expand=True)
accidents2.dateonly.head()

/home/pres/kamilimuds/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


1    11/09/2011
2    01/01/2011
3    01/01/2011
4    01/30/2011
5    01/01/2011
Name: dateonly, dtype: object

We then delete the columns we don't need using the drop method as show below

In [54]:
accidents2 = accidents2.drop("deleteall", axis=1)
accidents2.columns

Index(['Serial', 'Event', 'Code_County', 'County', 'Code_District', 'District',
       'Code_Division', 'Division', 'Date_YMD', 'Comments', 'Cause',
       'Description_of_Cause', 'Source', 'Magnitude', 'GLIDEnumber',
       'Other_sectors', 'DataCards', 'Deaths', 'Injured', 'Missing',
       'Houses_Destroyed', 'Houses_Damaged', 'Victims', 'Affected',
       'Relocated', 'Evacuated', 'Losses_$USD', 'Losses_$Local',
       'Education_centers', 'Hospitals', 'Damages_in_crops_Ha', 'Lost_Cattle',
       'OBJECTID', 'dateonly'],
      dtype='object')

Once we have dates we change them into the right format using to_datetime function as below

In [55]:
accidents2["date"] = pd.to_datetime(accidents2.dateonly, format="%m/%d/%Y")
accidents2.date.head()

1   2011-11-09
2   2011-01-01
3   2011-01-01
4   2011-01-30
5   2011-01-01
Name: date, dtype: datetime64[ns]

We decided to find out the regions in which there are the most accidents in any given month. However, we are alive to the fact that school holidays have very high population of travellers, we thus decided to remove the months of April, Augusta and December from our analysis so that we can remove the noisiness we assume they provide.

The first step if to separate months from the date variable as shown below.

In [70]:
accidents2[["month","therest"]] = accidents2.dateonly.str.split("/", 1, expand=True)
accidents2.month.describe()

count     1354
unique      12
top         09
freq       202
Name: month, dtype: object

Once we have done that we create a new dataframe that removes the months we intended to remove as shown below:

In [76]:
accidents3 = accidents2[-accidents2.month.isin(["04","08","12"])]

In [79]:
accidents3.columns

Index(['Serial', 'Event', 'Code_County', 'County', 'Code_District', 'District',
       'Code_Division', 'Division', 'Date_YMD', 'Comments', 'Cause',
       'Description_of_Cause', 'Source', 'Magnitude', 'GLIDEnumber',
       'Other_sectors', 'DataCards', 'Deaths', 'Injured', 'Missing',
       'Houses_Destroyed', 'Houses_Damaged', 'Victims', 'Affected',
       'Relocated', 'Evacuated', 'Losses_$USD', 'Losses_$Local',
       'Education_centers', 'Hospitals', 'Damages_in_crops_Ha', 'Lost_Cattle',
       'OBJECTID', 'dateonly', 'date', 'month', 'therest'],
      dtype='object')

We convert the death columns into numerals and find the mean number of deaths per day as show below:

We then arrange the deaths in descending order to find the table below:

In [83]:
# type(accidents3.Deaths[1])
accidents3.Deaths = pd.to_numeric(accidents3.Deaths)
type(accidents3.Deaths[1])

numpy.int64

In [93]:
accidents_regfreq = accidents3.groupby(["month","County"])\
    ["Deaths"]\
    .mean()\
    .reset_index()\
    .sort_values(by="Deaths", ascending=False)
accidents_regfreq

,month,County,Deaths
68,02,UASIN-GISHU,6.000000
67,02,TRANS-NZOIA,6.000000
221,09,TURKANA,6.000000
218,09,TAITA-TAVETA,4.333333
203,09,MACHAKOS,3.636364
206,09,MERU,3.333333
69,02,VIHIGA,3.000000
189,09,HOMABAY,3.000000
36,01,WEST POKOT,3.000000
193,09,KEIYO-MARAKWET,3.000000
